In [61]:
# Import các thư viện cần thiết
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [62]:
# Đường dẫn dữ liệu
raw_data_dir = "../data/new_data/raw/"
processed_data_dir = "../data/new_data/processed/"
os.makedirs(processed_data_dir, exist_ok=True)

# Danh sách các sàn giao dịch (giả định có trong thư mục raw)
exchanges = ["binance", "coinbase", "kucoin", "kraken"]


In [63]:
# Hợp nhất dữ liệu từ các file raw
all_data = []

for exchange in exchanges:
    exchange_dir = os.path.join(raw_data_dir, exchange)
    files = [f for f in os.listdir(exchange_dir) if f.endswith(".csv")]

    for file in files:
        file_path = os.path.join(exchange_dir, file)
        df = pd.read_csv(file_path)
        
        all_data.append(df)

# Gộp tất cả dữ liệu
new_data = pd.concat(all_data, ignore_index=True)
print("Dữ liệu sau khi hợp nhất:")
print(new_data.head())


Dữ liệu sau khi hợp nhất:
                  time     open     high      low    close     volume  \
0  2024-10-18 15:00:00  0.03861  0.03872  0.03853  0.03861  2005.2370   
1  2024-10-18 16:00:00  0.03860  0.03862  0.03844  0.03855  1241.2893   
2  2024-10-18 17:00:00  0.03854  0.03855  0.03845  0.03852   798.4479   
3  2024-10-18 18:00:00  0.03853  0.03880  0.03851  0.03867  2854.0002   
4  2024-10-18 19:00:00  0.03867  0.03872  0.03860  0.03865   813.1840   

      pair exchange  
0  ETH-BTC  Binance  
1  ETH-BTC  Binance  
2  ETH-BTC  Binance  
3  ETH-BTC  Binance  
4  ETH-BTC  Binance  


In [64]:
# Loại bỏ giá trị trùng lặp
new_data.drop_duplicates(subset=["time", "pair", "exchange"], inplace=True)

# Kiểm tra giá trị thiếu
missing_values = new_data.isnull().sum()
print("Giá trị thiếu:\n", missing_values)

# Điền giá trị thiếu bằng Interpolation
new_data.interpolate(method="linear", inplace=True)

# Xác nhận không còn giá trị thiếu
print("Giá trị thiếu sau khi xử lý:\n", new_data.isnull().sum())


Giá trị thiếu:
 time        0
open        0
high        0
low         0
close       0
volume      0
pair        0
exchange    0
dtype: int64
Giá trị thiếu sau khi xử lý:
 time        0
open        0
high        0
low         0
close       0
volume      0
pair        0
exchange    0
dtype: int64


/var/folders/rz/mr63129s7vs6092h_t0czm140000gn/T/ipykernel_6837/2372838812.py:9: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  new_data.interpolate(method="linear", inplace=True)


In [65]:
# Tính các đặc trưng mới
new_data["Price_Change_Pct"] = (new_data["close"] - new_data["open"]) / new_data["open"] * 100
new_data["Volume_Spike"] = new_data["volume"] / new_data["volume"].rolling(window=10).mean()
new_data["Volatility"] = new_data["high"] - new_data["low"]

# Điền NaN trong các cột mới (do rolling)
new_data.fillna(0, inplace=True)

print("Dữ liệu với đặc trưng mới:")
print(new_data[["Price_Change_Pct", "Volume_Spike", "Volatility"]].head())


Dữ liệu với đặc trưng mới:
   Price_Change_Pct  Volume_Spike  Volatility
0          0.000000           0.0     0.00019
1         -0.129534           0.0     0.00018
2         -0.051894           0.0     0.00010
3          0.363353           0.0     0.00029
4         -0.051720           0.0     0.00012


In [66]:
# Chuẩn hóa các cột đặc trưng
scaler = StandardScaler()
scaled_features = scaler.fit_transform(new_data[["open", "high", "low", "close", "volume", "Price_Change_Pct", "Volume_Spike", "Volatility"]])

# Tạo DataFrame mới cho dữ liệu chuẩn hóa
new_data_scaled = pd.DataFrame(scaled_features, columns=["Open", "High", "Low", "Close", "Volume", "Price_Change_Pct", "Volume_Spike", "Volatility"])

# Thêm lại các cột quan trọng
new_data_scaled["Timestamp"] = new_data["time"].values
new_data_scaled["Pair"] = new_data["pair"].values
new_data_scaled["Exchange"] = new_data["exchange"].values

# Kiểm tra kết quả
print("Dữ liệu đã chuẩn hóa:")
print(new_data_scaled.head())


Dữ liệu đã chuẩn hóa:
       Open      High       Low     Close    Volume  Price_Change_Pct  \
0 -0.117979 -0.117985 -0.117972 -0.117979 -0.131174         -0.002000   
1 -0.117979 -0.117985 -0.117972 -0.117979 -0.131382         -0.100156   
2 -0.117979 -0.117985 -0.117972 -0.117979 -0.131503         -0.041323   
3 -0.117979 -0.117985 -0.117972 -0.117979 -0.130943          0.273339   
4 -0.117979 -0.117985 -0.117972 -0.117979 -0.131499         -0.041191   

   Volume_Spike  Volatility            Timestamp     Pair Exchange  
0     -0.798094   -0.100947  2024-10-18 15:00:00  ETH-BTC  Binance  
1     -0.798094   -0.100947  2024-10-18 16:00:00  ETH-BTC  Binance  
2     -0.798094   -0.100949  2024-10-18 17:00:00  ETH-BTC  Binance  
3     -0.798094   -0.100944  2024-10-18 18:00:00  ETH-BTC  Binance  
4     -0.798094   -0.100949  2024-10-18 19:00:00  ETH-BTC  Binance  


In [67]:
# Sắp xếp lại các cột theo thứ tự mong muốn
new_data_scaled = new_data_scaled[[
    "Timestamp", "Open", "High", "Low", "Close", "Volume",
    "Pair", "Exchange", "Price_Change_Pct", "Volume_Spike", "Volatility"
]]


In [69]:
# Sắp xếp dữ liệu theo cột Timestamp
new_data_scaled = new_data_scaled.sort_values(by="Timestamp", ascending=True)

In [70]:
# Lưu dữ liệu đã xử lý
processed_file_path = os.path.join(processed_data_dir, "new_data_processed.csv")
new_data_scaled.to_csv(processed_file_path, index=False)
print(f"Dữ liệu đã được sắp xếp và lưu tại: {processed_file_path}")

Dữ liệu đã được sắp xếp và lưu tại: ../data/new_data/processed/new_data_processed.csv
